In [ ]:
from orbitfit.tle_utils import get_tle_lines, tle_list_to_tle_data, twoline2rv, wgs84
from orbitfit.utils import calculate_gps_tle_error
from orbitfit.rotate import rotate_gps
from orbitfit.io import parse_gps_data, GPS_state_vector_to_pandas_series

import datetime
import numpy as np
import pandas as pd
import os

%matplotlib notebook

In [ ]:
filename = 'Orekit_example_2/CH01_20210716_GPS_Test4_Rev6-3_rows_0to686_fixed.csv'

## Analyse status

In [ ]:
df_gps_raw = pd.read_csv(filename)
df_gps = df_gps_raw.apply(GPS_state_vector_to_pandas_series, axis=1)
df_gps = df_gps.set_index('date')
df_gps.status.plot()

In [ ]:
df_gps_locked = df_gps[df_gps.status == 1]
timediff = df_gps_locked.index.to_series().diff()
timediff = timediff[timediff > datetime.timedelta(minutes=5)]
timediff = timediff.apply(lambda x: x.total_seconds()/60)
timediff_sorted = timediff.sort_values(ascending=False)
timediff_sorted.head(10)

In [ ]:
df_gps = parse_gps_data(filename)
df_gps = df_gps.tz_localize(datetime.timezone.utc)

In [ ]:
df_gps_teme = rotate_gps(df_gps, method="E2T")

In [ ]:
tle_list = []
lines = ['1 44391U 19038E   21194.41242536  .00000328  00000-0  32780-4 0  9993',
         '2 44391  97.7766 159.4416 0018770 285.5407  74.3743 14.98090681110500']
tle_list.append(twoline2rv(*lines, whichconst=wgs84))
tle_list_with_data = tle_list_to_tle_data(tle_list)

In [ ]:
tle_list[0].epoch

In [ ]:
df_gps_tle_error = calculate_gps_tle_error(df_gps_teme, tle_list_with_data)

In [ ]:
df_gps_tle_error[['alongtrack_error_km', 'acrosstrack_error_km', 'radial_error_km']].plot(grid=True)

same 137 seconds delay between TLE and GPS

###  With time shift of 18sec

In [ ]:
df_gps.index = df_gps.index - datetime.timedelta(seconds=137/7.6)
df_gps_teme = rotate_gps(df_gps, method="E2T")
df_gps_tle_error = calculate_gps_tle_error(df_gps_teme, tle_list_with_data)
df_gps_tle_error[['alongtrack_error_km', 'acrosstrack_error_km', 'radial_error_km']].plot(grid=True)